In [ ]:
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

In [ ]:
!pip install langchain --upgrade
!pip install spotipy
!pip install replicate
!pip install python-dotenv

In [ ]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [ ]:
with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [ ]:
import spotipy.util as util
from langchain.requests import RequestsWrapper
from decouple import config

from dotenv import load_dotenv
load_dotenv()

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(raw_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']['scopes'].keys())
    access_token = util.prompt_for_user_token(scope=','.join(scopes))
    return {
        'Authorization': f'Bearer {access_token}'
    }

headers = construct_spotify_auth_headers(raw_spotify_api_spec)
print(headers)
requests_wrapper = RequestsWrapper(headers=headers)

In [ ]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import SerpAPIWrapper

llm = OpenAI(model_name="gpt-4", temperature=0.0)

spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)

search = SerpAPIWrapper()

tools = [
    Tool(
        name="SpotifyAPI",
        func=spotify_agent,
        description="Useful for getting information from the spotify api. pass the entire task to be completed in spotify, as this tool can support multiple steps"
    ),
    Tool(
        name="Google Search",
        func=search.run,
        description="Useful for searching Google for songs that fit a particular prompt, musical history, musical facts, and more"
    )
]

prefix = """You are a music assistant that helps create playlists. You have access to the following tools:"""
suffix = """Begin!

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=False)

user_input = input("Please enter your message: ")
agent_executor.run(user_input)